# A/B Testing Simulation

In this notebook, a user has a hidden preference within a single query. We use this to explore A/B testing. 

Of course, this problem needs to be multiplied over millions of queries!

1. The last judgments from chapter 11
2. Fully train a model w/ two feature sets (turn ch 11 auto LTR notebook into function) 
3. Simulate user interaction w/ rankings

In [1]:
import numpy as np
import pandas as pd

import requests
import sys
sys.path.append('..')
from ltr.client.solr_client import SolrClient

client = SolrClient(host='http://aips-solr:8983/solr')

In [2]:
def all_sessions():
    import glob
    sessions = pd.concat([pd.read_csv(f, compression='gzip')
                          for f in glob.glob('data/*_sessions.gz')])
    return sessions.rename(columns={'clicked_doc_id': 'doc_id'})
    
sessions = all_sessions()
sessions

ValueError: No objects to concatenate